# Data Preparation for Model Training

## Step 1: Confirm the raw2 structure

In [36]:
import os
import pandas as pd

# Define the directory containing the CSV files
directory = 'raw_data/'

# Initialize a list to hold the sets of column names from each CSV
columns_list = []

# Initialize a list to hold the paths of CSV files
csv_files = []

# Traverse the directory and collect all CSV file paths
for filename in os.listdir(directory):
    if filename.lower().endswith('.csv'):
        csv_files.append(os.path.join(directory, filename))

# Check if there are any CSV files in the directory
if not csv_files:
    print("No CSV files found in the specified directory.")
else:
    # Read the columns of each CSV file
    for file in csv_files:
        try:
            # Read only the header row to get column names
            df = pd.read_csv(file, nrows=0)
            columns_list.append(set(df.columns))
        except Exception as e:
            print(f"Error reading {file}: {e}")
            # Optionally, you can choose to exit or continue
            # exit()

    # Check if all CSV files have the same columns
    first_columns = columns_list[0]
    all_same = all(cols == first_columns for cols in columns_list)

    if not all_same:
        # Identify and list files with differing columns
        differing_files = [csv_files[i] for i, cols in enumerate(columns_list) if cols != first_columns]
        print("Not all CSV files have the same columns.")
        print("Files with differing columns:")
        for file in differing_files:
            print(f"- {file}")
    else:
        # Proceed to extract column names and 100th elements from one file
        sample_file = csv_files[0]
        try:
            df_sample = pd.read_csv(sample_file)
        except Exception as e:
            print(f"Error reading sample file {sample_file}: {e}")
            # exit()

        # Initialize a dictionary to store the 100th element of each column
        example_dict = {}
        for col in df_sample.columns:
            if len(df_sample[col]) >= 100:
                example_dict[col] = df_sample[col].iloc[1199]  # Zero-based index
            else:
                example_dict[col] = None  # Handle columns with fewer than 100 elements

        # Create a DataFrame from the dictionary for better readability
        example_df = pd.DataFrame(example_dict, index=['1200th Element'])

        # Display the DataFrame
        display(example_df)

,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,sr_lines,sr_lines_weights,...,RSI14,RSI7,RSI14_MA14,RSI7_MA7,MACD,MACD_Signal,MACD_Hist,BB_upper,BB_middle,BB_lower
1200th Element,2010-02-22 08:00:00+00:00,6.0,6.05,5.95,6.0,2257840,NaN,NaN,5.55;5.6;5.65;5.7;5.75;5.8;5.85;5.9;5.95;6.0;6...,2;9;7;2;10;3;4;5;6;8;10;3;6;10;1;5;11,...,55.555556,83.333333,29.421769,54.761905,-0.013243,-0.028234,0.014991,6.04785,5.8975,5.74715


In [37]:
# Initialize the NOTES dictionary
NOTES = {}

# Process 'Pivot Highs' and 'Pivot Lows' columns
for col in ['Pivot Highs', 'Pivot Lows']:
    if col in df_sample.columns:
        # Convert to binary: 1 if not NaN, 0 if NaN
        df_sample[col] = df_sample[col].apply(lambda x: 1 if pd.notna(x) else 0)
        
        # Add a note for the columns
        NOTES['pivot values'] = 'data leak'
        
# Display the modified DataFrame to verify the change
display(df_sample.head())
print(NOTES)

# Create the list of not_dealt columns, excluding 'Pivot Highs' and 'Pivot Lows'
not_dealt = [col for col in df_sample.columns if col not in ['Pivot Highs', 'Pivot Lows']]

# Print the not_dealt list to verify
print("Columns not yet dealt with:", not_dealt)

# Calculate percentage difference for BB columns
for col in ['BB_upper', 'BB_middle', 'BB_lower']:
    if col in df_sample.columns:
        df_sample[col] = (df_sample[col] - df_sample['close']) / df_sample['close']
        # Remove column from not_dealt
        not_dealt.remove(col)
        # Add note to NOTES dictionary
        NOTES[col] = 'data leak'

# Calculate percentage difference for EMA columns
for col in ['EMA20', 'EMA50', 'EMA100', 'EMA200']:
    if col in df_sample.columns and col in not_dealt: # Added check to see if the column is already processed
        df_sample[col] = (df_sample[col] - df_sample['close']) / df_sample['close']
        # Remove column from not_dealt
        not_dealt.remove(col)
        # Add note to NOTES dictionary
        NOTES[col] = 'data leak'

# Display the modified DataFrame to verify changes
display(df_sample.head())

# Print the updated not_dealt list and notes dictionary
print("Columns not yet dealt with:", not_dealt)
print("Updated NOTES dictionary:", NOTES)


,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,sr_lines,sr_lines_weights,...,RSI14,RSI7,RSI14_MA14,RSI7_MA7,MACD,MACD_Signal,MACD_Hist,BB_upper,BB_middle,BB_lower
0,2009-08-10 14:00:00+00:00,5.85,5.85,5.85,5.85,42,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,2009-08-11 06:00:00+00:00,5.80,5.85,5.70,5.70,116078,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.011966,-0.002393,-0.009573,NaN,NaN,NaN
2,2009-08-11 07:00:00+00:00,5.70,5.70,5.60,5.65,1233378,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.025193,-0.006953,-0.018240,NaN,NaN,NaN
3,2009-08-11 08:00:00+00:00,5.65,5.65,5.60,5.65,803878,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.035269,-0.012616,-0.022653,NaN,NaN,NaN
4,2009-08-11 09:00:00+00:00,5.65,5.65,5.65,5.65,252866,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.042761,-0.018645,-0.024116,NaN,NaN,NaN


{'pivot values': 'data leak'}
Columns not yet dealt with: ['time', 'open', 'high', 'low', 'close', 'volume', 'sr_lines', 'sr_lines_weights', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'Valid Pivots', 'TrendLines', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_upper', 'BB_middle', 'BB_lower']


,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,sr_lines,sr_lines_weights,...,RSI14,RSI7,RSI14_MA14,RSI7_MA7,MACD,MACD_Signal,MACD_Hist,BB_upper,BB_middle,BB_lower
0,2009-08-10 14:00:00+00:00,5.85,5.85,5.85,5.85,42,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,2009-08-11 06:00:00+00:00,5.80,5.85,5.70,5.70,116078,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.011966,-0.002393,-0.009573,NaN,NaN,NaN
2,2009-08-11 07:00:00+00:00,5.70,5.70,5.60,5.65,1233378,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.025193,-0.006953,-0.018240,NaN,NaN,NaN
3,2009-08-11 08:00:00+00:00,5.65,5.65,5.60,5.65,803878,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.035269,-0.012616,-0.022653,NaN,NaN,NaN
4,2009-08-11 09:00:00+00:00,5.65,5.65,5.65,5.65,252866,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Columns not yet dealt with: ['time', 'open', 'high', 'low', 'close', 'volume', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'Valid Pivots', 'TrendLines', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist']
Updated NOTES dictionary: {'pivot values': 'data leak', 'BB_upper': 'data leak', 'BB_middle': 'data leak', 'BB_lower': 'data leak', 'EMA20': 'data leak', 'EMA50': 'data leak', 'EMA100': 'data leak', 'EMA200': 'data leak'}


In [38]:
# Create a new NOTES dictionary with the desired structure
new_notes = {}
new_notes['data leak'] = [key for key in NOTES]

# Update the NOTES dictionary
NOTES = new_notes

# Print the updated NOTES dictionary
print("Updated NOTES dictionary:", NOTES)

Updated NOTES dictionary: {'data leak': ['pivot values', 'BB_upper', 'BB_middle', 'BB_lower', 'EMA20', 'EMA50', 'EMA100', 'EMA200']}


In [39]:
# Check if df_sample has at least 400 rows
if len(df_sample) >= 400:
    # Print the 400th element (row) including all columns
    print("400th element of df_sample:")
    display(df_sample.iloc[[399]]) # Use display for better readability
else:
    print("df_sample does not have 400 rows. Cannot display the 400th element.")

400th element of df_sample:


,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,sr_lines,sr_lines_weights,...,RSI14,RSI7,RSI14_MA14,RSI7_MA7,MACD,MACD_Signal,MACD_Hist,BB_upper,BB_middle,BB_lower
399,2009-10-14 08:00:00+00:00,5.75,5.75,5.7,5.75,942670,0,0,5.4;5.45;5.5;5.55;5.6;5.65;5.7;5.75;5.8;5.85;5...,8;6;3;2;4;10;14;11;20;20;14;10;3,...,50.0,50.0,44.84127,42.857143,-0.015186,-0.014391,-0.000795,0.012836,-0.00087,-0.014575


In [40]:
import numpy as np
def extract_sr_features(row):
    """Extracts and returns support/resistance features from a row."""
    
    close_price = row['close']
    sr_lines_str = row['sr_lines']
    sr_weights_str = row['sr_lines_weights']

    if pd.isna(sr_lines_str) or pd.isna(sr_weights_str):
        return [np.nan] * 13  # Return NaNs if either string is NaN

    sr_lines = [float(x) for x in sr_lines_str.split(';')]
    sr_weights = [int(x) for x in sr_weights_str.split(';')]
    
    # Combine lines and weights into tuples, handle cases where lengths don't match
    sr_pairs = list(zip(sr_lines, sr_weights))
    
    # If sr_pairs is empty, return NaNs
    if not sr_pairs:
        return [np.nan] * 13

    # Separate lines above and below the close price
    below_lines = [(line, weight) for line, weight in sr_pairs if line < close_price]
    above_lines = [(line, weight) for line, weight in sr_pairs if line > close_price]
    
    # Get lines with highest weights
    highest_weight_lines = sorted(sr_pairs, key=lambda x: x[1], reverse=True)
    
    # Calculate the number of lines and total weights below and above the close price
    num_sr_below = len(below_lines)
    num_sr_above = len(above_lines)
    total_weight_below = sum(weight for _, weight in below_lines) if below_lines else 0
    total_weight_above = sum(weight for _, weight in above_lines) if above_lines else 0

    # Extract top 3 from each category, handling cases where there are less than 3
    below_sr_features = [line for line, _ in sorted(below_lines,key=lambda x: (close_price - x[0]))[:3]]
    above_sr_features = [line for line, _ in sorted(above_lines,key=lambda x: (x[0] - close_price))[:3]]
    highest_weight_features = [line for line, _ in highest_weight_lines[:3]]
    
    # Pad lists with NaNs if they have less than 3 elements
    while len(below_sr_features) < 3:
       below_sr_features.append(np.nan)
    while len(above_sr_features) < 3:
       above_sr_features.append(np.nan)
    while len(highest_weight_features) < 3:
       highest_weight_features.append(np.nan)

    return below_sr_features + above_sr_features + highest_weight_features + [num_sr_below, num_sr_above, total_weight_below, total_weight_above]

# Apply the function to create new columns
sr_features = df_sample.apply(extract_sr_features, axis=1, result_type='expand')
sr_features.columns = [
    'sr_below_1', 'sr_below_2', 'sr_below_3', 
    'sr_above_1', 'sr_above_2', 'sr_above_3',
    'sr_weight_1', 'sr_weight_2', 'sr_weight_3',
    'num_sr_below', 'num_sr_above', 'total_weight_below', 'total_weight_above'
]

# Combine the new features with the original dataframe
df_sample = pd.concat([df_sample, sr_features], axis=1)

# Remove sr_lines and sr_lines_weights from not_dealt
if 'sr_lines' in not_dealt:
    not_dealt.remove('sr_lines')
if 'sr_lines_weights' in not_dealt:
    not_dealt.remove('sr_lines_weights')
    
# Add sr_lines and sr_lines_weights to data leak
NOTES['data leak'] += ['sr_lines', 'sr_lines_weights']

# Display the head of the modified dataframe and updated not_dealt and notes
display(df_sample.head())
print("Columns not yet dealt with:", not_dealt)
print("Updated NOTES dictionary:", NOTES)

,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,sr_lines,sr_lines_weights,...,sr_above_1,sr_above_2,sr_above_3,sr_weight_1,sr_weight_2,sr_weight_3,num_sr_below,num_sr_above,total_weight_below,total_weight_above
0,2009-08-10 14:00:00+00:00,5.85,5.85,5.85,5.85,42,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-08-11 06:00:00+00:00,5.80,5.85,5.70,5.70,116078,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-08-11 07:00:00+00:00,5.70,5.70,5.60,5.65,1233378,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-08-11 08:00:00+00:00,5.65,5.65,5.60,5.65,803878,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-08-11 09:00:00+00:00,5.65,5.65,5.65,5.65,252866,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Columns not yet dealt with: ['time', 'open', 'high', 'low', 'close', 'volume', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'Valid Pivots', 'TrendLines', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist']
Updated NOTES dictionary: {'data leak': ['pivot values', 'BB_upper', 'BB_middle', 'BB_lower', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'sr_lines', 'sr_lines_weights']}


In [41]:
# Drop the 'sr_lines' and 'sr_lines_weights' columns from the DataFrame
df_sample = df_sample.drop(columns=['sr_lines', 'sr_lines_weights'])

# Display the first few rows of the modified DataFrame
display(df_sample.head())

# Print the updated list of columns
print("Updated Columns:", df_sample.columns.tolist())

,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,Pivot High Mistaken,Pivot Low Mistaken,...,sr_above_1,sr_above_2,sr_above_3,sr_weight_1,sr_weight_2,sr_weight_3,num_sr_below,num_sr_above,total_weight_below,total_weight_above
0,2009-08-10 14:00:00+00:00,5.85,5.85,5.85,5.85,42,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-08-11 06:00:00+00:00,5.80,5.85,5.70,5.70,116078,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-08-11 07:00:00+00:00,5.70,5.70,5.60,5.65,1233378,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-08-11 08:00:00+00:00,5.65,5.65,5.60,5.65,803878,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-08-11 09:00:00+00:00,5.65,5.65,5.65,5.65,252866,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Updated Columns: ['time', 'open', 'high', 'low', 'close', 'volume', 'Pivot Highs', 'Pivot Lows', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'Valid Pivots', 'TrendLines', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_upper', 'BB_middle', 'BB_lower', 'sr_below_1', 'sr_below_2', 'sr_below_3', 'sr_above_1', 'sr_above_2', 'sr_above_3', 'sr_weight_1', 'sr_weight_2', 'sr_weight_3', 'num_sr_below', 'num_sr_above', 'total_weight_below', 'total_weight_above']


In [42]:
# Check if df_sample has at least 1000 rows
if len(df_sample) >= 1000:
    # Get the 1000th row (index 999)
    row = df_sample.iloc[999]

    # Print the column names and their corresponding values
    print("Column Names and Values for 1000th Row:")
    for col_name, value in row.items():
        print(f"{col_name}: {value}")
else:
    print("df_sample does not have 1000 rows. Cannot display the 1000th row.")

Column Names and Values for 1000th Row:
time: 2010-01-20 15:00:00+00:00
open: 6.05
high: 6.1
low: 5.95
close: 6.05
volume: 4902793
Pivot Highs: 0
Pivot Lows: 0
Pivot High Mistaken: False
Pivot Low Mistaken: False
Valid Pivots: high@2009-12-03T12:00:00+00:00@5.65@3;low@2009-12-04T08:00:00+00:00@5.5@1;high@2009-12-08T08:00:00+00:00@5.7@4;low@2009-12-08T14:00:00+00:00@5.45@1;high@2009-12-09T12:00:00+00:00@5.6@3;low@2009-12-10T07:00:00+00:00@5.45@1;high@2009-12-10T13:00:00+00:00@5.6@2;low@2009-12-11T14:00:00+00:00@5.4@2;low@2009-12-14T12:00:00+00:00@5.45@3;high@2009-12-14T14:00:00+00:00@5.6@1;low@2009-12-15T12:00:00+00:00@5.45@2;high@2009-12-16T08:00:00+00:00@5.65@1;low@2009-12-17T15:00:00+00:00@5.4@3;high@2009-12-21T14:00:00+00:00@5.55@3;high@2009-12-22T07:00:00+00:00@5.6@4;low@2009-12-22T13:00:00+00:00@5.5@2;low@2009-12-23T12:00:00+00:00@5.5@2;high@2009-12-28T07:00:00+00:00@5.65@5;low@2009-12-29T07:00:00+00:00@5.5@2;low@2009-12-30T09:00:00+00:00@5.5@3;high@2009-12-31T10:00:00+00:00@5.75@

In [43]:
# List of columns to process
sr_columns = [
    'sr_below_1', 'sr_below_2', 'sr_below_3',
    'sr_above_1', 'sr_above_2', 'sr_above_3',
    'sr_weight_1', 'sr_weight_2', 'sr_weight_3'
]

# Iterate through the columns and calculate the percentage difference
for col in sr_columns:
    if col in df_sample.columns:
        df_sample[col] = df_sample.apply(
            lambda row: (row[col] - row['close']) / row['close'] if pd.notna(row[col]) else row[col],
            axis=1
        )

# Display the modified DataFrame's head to verify changes
display(df_sample.head())

# Print a message to show the completion of changes
print("Support/Resistance columns updated to percentage difference values.")

,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,Pivot High Mistaken,Pivot Low Mistaken,...,sr_above_1,sr_above_2,sr_above_3,sr_weight_1,sr_weight_2,sr_weight_3,num_sr_below,num_sr_above,total_weight_below,total_weight_above
0,2009-08-10 14:00:00+00:00,5.85,5.85,5.85,5.85,42,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-08-11 06:00:00+00:00,5.80,5.85,5.70,5.70,116078,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-08-11 07:00:00+00:00,5.70,5.70,5.60,5.65,1233378,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-08-11 08:00:00+00:00,5.65,5.65,5.60,5.65,803878,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-08-11 09:00:00+00:00,5.65,5.65,5.65,5.65,252866,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Support/Resistance columns updated to percentage difference values.


In [44]:
# Check if df_sample has at least 1000 rows
if len(df_sample) >= 1000:
    # Get the 1000th row (index 999)
    row = df_sample.iloc[999]

    # Print the column names and their corresponding values
    print("Column Names and Values for 1000th Row:")
    for col_name, value in row.items():
        print(f"{col_name}: {value}")
else:
    print("df_sample does not have 1000 rows. Cannot display the 1000th row.")

Column Names and Values for 1000th Row:
time: 2010-01-20 15:00:00+00:00
open: 6.05
high: 6.1
low: 5.95
close: 6.05
volume: 4902793
Pivot Highs: 0
Pivot Lows: 0
Pivot High Mistaken: False
Pivot Low Mistaken: False
Valid Pivots: high@2009-12-03T12:00:00+00:00@5.65@3;low@2009-12-04T08:00:00+00:00@5.5@1;high@2009-12-08T08:00:00+00:00@5.7@4;low@2009-12-08T14:00:00+00:00@5.45@1;high@2009-12-09T12:00:00+00:00@5.6@3;low@2009-12-10T07:00:00+00:00@5.45@1;high@2009-12-10T13:00:00+00:00@5.6@2;low@2009-12-11T14:00:00+00:00@5.4@2;low@2009-12-14T12:00:00+00:00@5.45@3;high@2009-12-14T14:00:00+00:00@5.6@1;low@2009-12-15T12:00:00+00:00@5.45@2;high@2009-12-16T08:00:00+00:00@5.65@1;low@2009-12-17T15:00:00+00:00@5.4@3;high@2009-12-21T14:00:00+00:00@5.55@3;high@2009-12-22T07:00:00+00:00@5.6@4;low@2009-12-22T13:00:00+00:00@5.5@2;low@2009-12-23T12:00:00+00:00@5.5@2;high@2009-12-28T07:00:00+00:00@5.65@5;low@2009-12-29T07:00:00+00:00@5.5@2;low@2009-12-30T09:00:00+00:00@5.5@3;high@2009-12-31T10:00:00+00:00@5.75@

In [45]:
def extract_trendline_features(row):
    """Extracts and returns trendline features from a row."""
    
    close_price = row['close']
    trendlines_str = row['TrendLines']

    if pd.isna(trendlines_str):
        return [np.nan] * 13  # Return NaNs if string is NaN

    # Split the trendlines string and parse each line
    trendlines = []
    for tl in trendlines_str.split('|'):
        direction, price, weight = tl.split('@')
        trendlines.append((direction, float(price), int(weight)))
    
    # Separate lines above and below the close price
    below_lines = [(price, weight) for direction, price, weight in trendlines 
                  if price < close_price]
    above_lines = [(price, weight) for direction, price, weight in trendlines 
                  if price > close_price]
    
    # Sort by distance from close price
    below_lines.sort(key=lambda x: (close_price - x[0]))
    above_lines.sort(key=lambda x: (x[0] - close_price))
    
    # Sort by weight for highest weight features
    all_lines = [(price, weight) for direction, price, weight in trendlines]
    highest_weight_lines = sorted(all_lines, key=lambda x: x[1], reverse=True)
    
    # Calculate statistics
    num_tl_below = len(below_lines)
    num_tl_above = len(above_lines)
    total_weight_below = sum(weight for _, weight in below_lines) if below_lines else 0
    total_weight_above = sum(weight for _, weight in above_lines) if above_lines else 0
    
    # Extract features (using percentage difference from close)
    below_features = [(price - close_price) / close_price 
                     for price, _ in below_lines[:3]]
    above_features = [(price - close_price) / close_price 
                     for price, _ in above_lines[:3]]
    weight_features = [(price - close_price) / close_price 
                      for price, _ in highest_weight_lines[:3]]
    
    # Pad lists with NaNs if they have less than 3 elements
    while len(below_features) < 3:
        below_features.append(np.nan)
    while len(above_features) < 3:
        above_features.append(np.nan)
    while len(weight_features) < 3:
        weight_features.append(np.nan)
    
    return below_features + above_features + weight_features + [
        num_tl_below, num_tl_above, total_weight_below, total_weight_above
    ]

# Apply the function to create new columns
tl_features = df_sample.apply(extract_trendline_features, axis=1, result_type='expand')
tl_features.columns = [
    'tl_below_1', 'tl_below_2', 'tl_below_3',
    'tl_above_1', 'tl_above_2', 'tl_above_3',
    'tl_weight_1', 'tl_weight_2', 'tl_weight_3',
    'num_tl_below', 'num_tl_above', 'total_tl_weight_below', 'total_tl_weight_above'
]

# Combine the new features with the original dataframe
df_sample = pd.concat([df_sample, tl_features], axis=1)

# Drop the original columns that are no longer needed
df_sample = df_sample.drop(columns=['Valid Pivots', 'TrendLines'])

# Add to NOTES
NOTES['data leak'] += ['Valid Pivots', 'TrendLines']

# Display sample to verify changes
display(df_sample.head())
print("\nColumns after processing:", df_sample.columns.tolist())
print("\nUpdated NOTES dictionary:", NOTES)

,time,open,high,low,close,volume,Pivot Highs,Pivot Lows,Pivot High Mistaken,Pivot Low Mistaken,...,tl_above_1,tl_above_2,tl_above_3,tl_weight_1,tl_weight_2,tl_weight_3,num_tl_below,num_tl_above,total_tl_weight_below,total_tl_weight_above
0,2009-08-10 14:00:00+00:00,5.85,5.85,5.85,5.85,42,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-08-11 06:00:00+00:00,5.80,5.85,5.70,5.70,116078,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-08-11 07:00:00+00:00,5.70,5.70,5.60,5.65,1233378,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-08-11 08:00:00+00:00,5.65,5.65,5.60,5.65,803878,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-08-11 09:00:00+00:00,5.65,5.65,5.65,5.65,252866,0,0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Columns after processing: ['time', 'open', 'high', 'low', 'close', 'volume', 'Pivot Highs', 'Pivot Lows', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_upper', 'BB_middle', 'BB_lower', 'sr_below_1', 'sr_below_2', 'sr_below_3', 'sr_above_1', 'sr_above_2', 'sr_above_3', 'sr_weight_1', 'sr_weight_2', 'sr_weight_3', 'num_sr_below', 'num_sr_above', 'total_weight_below', 'total_weight_above', 'tl_below_1', 'tl_below_2', 'tl_below_3', 'tl_above_1', 'tl_above_2', 'tl_above_3', 'tl_weight_1', 'tl_weight_2', 'tl_weight_3', 'num_tl_below', 'num_tl_above', 'total_tl_weight_below', 'total_tl_weight_above']

Updated NOTES dictionary: {'data leak': ['pivot values', 'BB_upper', 'BB_middle', 'BB_lower', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'sr_lines', 'sr_lines_weights', 'Valid Pivots', 'TrendLines']}


In [46]:
def print_column_categories(df, row_index=999):
    """Print columns grouped by their types with example values."""
    
    row = df.iloc[row_index]
    
    # Define column categories
    categories = {
        'Time': ['time'],
        'Price Data': ['open', 'high', 'low', 'close', 'volume'],
        'Pivot Information': ['Pivot Highs', 'Pivot Lows', 'Pivot High Mistaken', 'Pivot Low Mistaken'],
        'Technical Indicators': ['EMA20', 'EMA50', 'EMA100', 'EMA200', 
                               'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7',
                               'MACD', 'MACD_Signal', 'MACD_Hist',
                               'BB_upper', 'BB_middle', 'BB_lower'],
        'Support/Resistance Features': ['sr_below_1', 'sr_below_2', 'sr_below_3',
                                      'sr_above_1', 'sr_above_2', 'sr_above_3',
                                      'sr_weight_1', 'sr_weight_2', 'sr_weight_3',
                                      'num_sr_below', 'num_sr_above',
                                      'total_weight_below', 'total_weight_above'],
        'Trendline Features': ['tl_below_1', 'tl_below_2', 'tl_below_3',
                              'tl_above_1', 'tl_above_2', 'tl_above_3',
                              'tl_weight_1', 'tl_weight_2', 'tl_weight_3',
                              'num_tl_below', 'num_tl_above',
                              'total_tl_weight_below', 'total_tl_weight_above']
    }
    
    # Print each category
    for category, cols in categories.items():
        print(f"\n{category}:")
        print("-" * len(category))
        for col in cols:
            if col in df.columns:
                print(f"{col:<20}: {row[col]}")
            else:
                print(f"{col:<20}: Not in DataFrame")
        print()

# Print the categorized columns with their values
print("Data Overview (1000th row):")
print("=" * 50)
print_column_categories(df_sample)

# Print summary of data leakage concerns
print("\nData Leakage Concerns:")
print("=" * 50)
for category, items in NOTES.items():
    print(f"\n{category}:")
    for item in items:
        print(f"- {item}")

Data Overview (1000th row):

Time:
----
time                : 2010-01-20 15:00:00+00:00


Price Data:
----------
open                : 6.05
high                : 6.1
low                 : 5.95
close               : 6.05
volume              : 4902793


Pivot Information:
-----------------
Pivot Highs         : 0
Pivot Lows          : 0
Pivot High Mistaken : False
Pivot Low Mistaken  : False


Technical Indicators:
--------------------
EMA20               : -0.031025232577571955
EMA50               : -0.045517311936884096
EMA100              : -0.05311352139002547
EMA200              : -0.062219652033990526
RSI14               : 76.92307692307696
RSI7                : 83.33333333333339
RSI14_MA14          : 61.887537066108514
RSI7_MA7            : 85.2040816326531
MACD                : 0.0915011236615175
MACD_Signal         : 0.0616662014971419
MACD_Hist           : 0.0298349221643755
BB_upper            : 0.01101456636610095
BB_middle           : -0.039256198347107266
BB_lower          

In [47]:
# Process time column to extract features
def extract_time_features(time_str):
    # Convert string to datetime
    dt = pd.to_datetime(time_str)
    
    # Add 2 hours to fix time distortion
    dt = dt + pd.Timedelta(hours=2)
    
    # Extract time features
    month = dt.month
    day = dt.day
    day_of_week = dt.dayofweek  # Monday=0, Sunday=6
    hour = dt.hour
    
    # Calculate season (meteorological seasons)
    # Spring: 3-5, Summer: 6-8, Autumn: 9-11, Winter: 12-2
    if month in [3, 4, 5]:
        season = 0  # Spring
    elif month in [6, 7, 8]:
        season = 1  # Summer
    elif month in [9, 10, 11]:
        season = 2  # Autumn
    else:
        season = 3  # Winter
    
    return pd.Series({
        'month': month,
        'day': day,
        'day_of_week': day_of_week,
        'hour': hour,
        'season': season
    })

# Apply time feature extraction
time_features = df_sample['time'].apply(extract_time_features)
df_sample = pd.concat([df_sample, time_features], axis=1)

# Drop original time column
df_sample = df_sample.drop(columns=['time'])

# Display sample and verification
print("\nVerifying time features for a sample row (index 999):")
sample_row = df_sample.iloc[999]
print(f"Month: {sample_row['month']}")
print(f"Day: {sample_row['day']}")
print(f"Day of Week: {sample_row['day_of_week']} (0=Monday, 6=Sunday)")
print(f"Hour: {sample_row['hour']}")
print(f"Season: {sample_row['season']} (0=Spring, 1=Summer, 2=Autumn, 3=Winter)")

# Display updated columns
print("\nUpdated column list:")
print(df_sample.columns.tolist())

print("\nUpdated NOTES dictionary:", NOTES)


Verifying time features for a sample row (index 999):
Month: 1
Day: 20
Day of Week: 2 (0=Monday, 6=Sunday)
Hour: 17
Season: 3 (0=Spring, 1=Summer, 2=Autumn, 3=Winter)

Updated column list:
['open', 'high', 'low', 'close', 'volume', 'Pivot Highs', 'Pivot Lows', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_upper', 'BB_middle', 'BB_lower', 'sr_below_1', 'sr_below_2', 'sr_below_3', 'sr_above_1', 'sr_above_2', 'sr_above_3', 'sr_weight_1', 'sr_weight_2', 'sr_weight_3', 'num_sr_below', 'num_sr_above', 'total_weight_below', 'total_weight_above', 'tl_below_1', 'tl_below_2', 'tl_below_3', 'tl_above_1', 'tl_above_2', 'tl_above_3', 'tl_weight_1', 'tl_weight_2', 'tl_weight_3', 'num_tl_below', 'num_tl_above', 'total_tl_weight_below', 'total_tl_weight_above', 'month', 'day', 'day_of_week', 'hour', 'season']

Updated NOTES dictionary: {'data leak': ['pivot values', 'BB_upper', 'BB_

In [48]:
# Check if df_sample has at least 1000 rows
if len(df_sample) >= 1000:
    # Get the 1000th row (index 999)
    row = df_sample.iloc[999]

    # Print the column names and their corresponding values
    print("Column Names and Values for 1000th Row:")
    for col_name, value in row.items():
        print(f"{col_name}: {value}")
else:
    print("df_sample does not have 1000 rows. Cannot display the 1000th row.")

Column Names and Values for 1000th Row:
open: 6.05
high: 6.1
low: 5.95
close: 6.05
volume: 4902793
Pivot Highs: 0
Pivot Lows: 0
Pivot High Mistaken: False
Pivot Low Mistaken: False
EMA20: -0.031025232577571955
EMA50: -0.045517311936884096
EMA100: -0.05311352139002547
EMA200: -0.062219652033990526
RSI14: 76.92307692307696
RSI7: 83.33333333333339
RSI14_MA14: 61.887537066108514
RSI7_MA7: 85.2040816326531
MACD: 0.0915011236615175
MACD_Signal: 0.0616662014971419
MACD_Hist: 0.0298349221643755
BB_upper: 0.01101456636610095
BB_middle: -0.039256198347107266
BB_lower: -0.08952696306031548
sr_below_1: -0.02479338842975198
sr_below_2: -0.03305785123966945
sr_below_3: -0.04132231404958678
sr_above_1: 0.008264462809917326
sr_above_2: nan
sr_above_3: nan
sr_weight_1: -0.07438016528925623
sr_weight_2: -0.06611570247933876
sr_weight_3: -0.049586776859504106
num_sr_below: 11.0
num_sr_above: 1.0
total_weight_below: 92.0
total_weight_above: 2.0
tl_below_1: -0.046280991735537236
tl_below_2: -0.047933884297

In [49]:
# Check if df_sample has at least 1000 rows
if len(df_sample) >= 1000:
    # Get the 1000th row (index 999)
    row = df_sample.iloc[999]

    # Print the column names and their corresponding values
    print("Column Names and Values for 1000th Row:")
    for col_name, value in row.items():
        print(f"{col_name}: {value}")
else:
    print("df_sample does not have 1000 rows. Cannot display the 1000th row.")

Column Names and Values for 1000th Row:
open: 6.05
high: 6.1
low: 5.95
close: 6.05
volume: 4902793
Pivot Highs: 0
Pivot Lows: 0
Pivot High Mistaken: False
Pivot Low Mistaken: False
EMA20: -0.031025232577571955
EMA50: -0.045517311936884096
EMA100: -0.05311352139002547
EMA200: -0.062219652033990526
RSI14: 76.92307692307696
RSI7: 83.33333333333339
RSI14_MA14: 61.887537066108514
RSI7_MA7: 85.2040816326531
MACD: 0.0915011236615175
MACD_Signal: 0.0616662014971419
MACD_Hist: 0.0298349221643755
BB_upper: 0.01101456636610095
BB_middle: -0.039256198347107266
BB_lower: -0.08952696306031548
sr_below_1: -0.02479338842975198
sr_below_2: -0.03305785123966945
sr_below_3: -0.04132231404958678
sr_above_1: 0.008264462809917326
sr_above_2: nan
sr_above_3: nan
sr_weight_1: -0.07438016528925623
sr_weight_2: -0.06611570247933876
sr_weight_3: -0.049586776859504106
num_sr_below: 11.0
num_sr_above: 1.0
total_weight_below: 92.0
total_weight_above: 2.0
tl_below_1: -0.046280991735537236
tl_below_2: -0.047933884297

In [50]:
# Calculate price changes relative to previous close
price_columns = ['open', 'high', 'low', 'close']
for col in price_columns:
    df_sample[f'{col}_pct'] = df_sample[col].pct_change()

# Calculate rolling volume averages
volume_periods = [7, 21, 50, 100, 200]
for period in volume_periods:
    df_sample[f'volume_ratio_{period}'] = (
        df_sample['volume'] / df_sample['volume'].rolling(window=period).mean()
    )

# Drop original price and volume columns
df_sample = df_sample.drop(columns=price_columns + ['volume'])

# Rename the new columns to simpler names
rename_dict = {
    'open_pct': 'open',
    'high_pct': 'high',
    'low_pct': 'low',
    'close_pct': 'close',
}
df_sample = df_sample.rename(columns=rename_dict)

# Display sample row to verify changes
print("\nSample row (index 999) after price and volume transformations:")
sample_row = df_sample.iloc[999]
print("\nPrice changes (%):")
for col in ['open', 'high', 'low', 'close']:
    print(f"{col:<15}: {sample_row[col]:.6f}")

print("\nVolume ratios:")
for period in volume_periods:
    print(f"vs {period:<3} avg    : {sample_row[f'volume_ratio_{period}']:.6f}")

# Display current columns
print("\nUpdated column list:")
print(df_sample.columns.tolist())


Sample row (index 999) after price and volume transformations:

Price changes (%):
open           : 0.000000
high           : 0.008264
low            : -0.008333
close          : 0.008333

Volume ratios:
vs 7   avg    : 1.262383
vs 21  avg    : 2.239980
vs 50  avg    : 3.354669
vs 100 avg    : 3.763748
vs 200 avg    : 5.916032

Updated column list:
['Pivot Highs', 'Pivot Lows', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_upper', 'BB_middle', 'BB_lower', 'sr_below_1', 'sr_below_2', 'sr_below_3', 'sr_above_1', 'sr_above_2', 'sr_above_3', 'sr_weight_1', 'sr_weight_2', 'sr_weight_3', 'num_sr_below', 'num_sr_above', 'total_weight_below', 'total_weight_above', 'tl_below_1', 'tl_below_2', 'tl_below_3', 'tl_above_1', 'tl_above_2', 'tl_above_3', 'tl_weight_1', 'tl_weight_2', 'tl_weight_3', 'num_tl_below', 'num_tl_above', 'total_tl_weight_below', 'total_tl_weight_above', 'mo

In [51]:
# Check if df_sample has at least 1000 rows
if len(df_sample) >= 1000:
    # Get the 1000th row (index 999)
    row = df_sample.iloc[999]

    # Print the column names and their corresponding values
    print("Column Names and Values for 1000th Row:")
    for col_name, value in row.items():
        print(f"{col_name}: {value}")
else:
    print("df_sample does not have 1000 rows. Cannot display the 1000th row.")

Column Names and Values for 1000th Row:
Pivot Highs: 0
Pivot Lows: 0
Pivot High Mistaken: False
Pivot Low Mistaken: False
EMA20: -0.031025232577571955
EMA50: -0.045517311936884096
EMA100: -0.05311352139002547
EMA200: -0.062219652033990526
RSI14: 76.92307692307696
RSI7: 83.33333333333339
RSI14_MA14: 61.887537066108514
RSI7_MA7: 85.2040816326531
MACD: 0.0915011236615175
MACD_Signal: 0.0616662014971419
MACD_Hist: 0.0298349221643755
BB_upper: 0.01101456636610095
BB_middle: -0.039256198347107266
BB_lower: -0.08952696306031548
sr_below_1: -0.02479338842975198
sr_below_2: -0.03305785123966945
sr_below_3: -0.04132231404958678
sr_above_1: 0.008264462809917326
sr_above_2: nan
sr_above_3: nan
sr_weight_1: -0.07438016528925623
sr_weight_2: -0.06611570247933876
sr_weight_3: -0.049586776859504106
num_sr_below: 11.0
num_sr_above: 1.0
total_weight_below: 92.0
total_weight_above: 2.0
tl_below_1: -0.046280991735537236
tl_below_2: -0.04793388429752067
tl_below_3: -0.052892561983470976
tl_above_1: 0.00826

In [52]:
# Create RSI-derived features
def calculate_rsi_features(df):
    """Calculate various RSI-derived features."""
    
    # Initialize new columns with RSI zones (0-30 oversold, 70-100 overbought)
    df['rsi14_zone'] = pd.cut(df['RSI14'], 
                             bins=[-float('inf'), 30, 70, float('inf')],
                             labels=[-1, 0, 1])  # -1: oversold, 0: neutral, 1: overbought
    
    df['rsi7_zone'] = pd.cut(df['RSI7'], 
                            bins=[-float('inf'), 30, 70, float('inf')],
                            labels=[-1, 0, 1])
    
    # RSI divergence from MA (normalized)
    df['rsi14_ma_div'] = (df['RSI14'] - df['RSI14_MA14']) / 100
    df['rsi7_ma_div'] = (df['RSI7'] - df['RSI7_MA7']) / 100
    
    # RSI momentum (change from previous period)
    df['rsi14_momentum'] = df['RSI14'].diff() / 100
    df['rsi7_momentum'] = df['RSI7'].diff() / 100
    
    # RSI cross signals (when RSI crosses its MA)
    df['rsi14_cross'] = np.where(
        (df['RSI14'].shift(1) < df['RSI14_MA14'].shift(1)) & 
        (df['RSI14'] > df['RSI14_MA14']), 1,
        np.where(
            (df['RSI14'].shift(1) > df['RSI14_MA14'].shift(1)) & 
            (df['RSI14'] < df['RSI14_MA14']), -1, 0
        )
    )
    
    df['rsi7_cross'] = np.where(
        (df['RSI7'].shift(1) < df['RSI7_MA7'].shift(1)) & 
        (df['RSI7'] > df['RSI7_MA7']), 1,
        np.where(
            (df['RSI7'].shift(1) > df['RSI7_MA7'].shift(1)) & 
            (df['RSI7'] < df['RSI7_MA7']), -1, 0
        )
    )
    
    # RSI agreement (whether RSI7 and RSI14 are in same zone)
    df['rsi_agreement'] = (df['rsi14_zone'] == df['rsi7_zone']).astype(int)
    
    # Drop original RSI columns as they're now transformed into more meaningful features
    df = df.drop(columns=['RSI14', 'RSI7', 'RSI14_MA14', 'RSI7_MA7'])
    
    return df

# Apply RSI transformations
df_sample = calculate_rsi_features(df_sample)

# Display sample row to verify changes
print("\nSample row (index 999) after RSI feature engineering:")
sample_row = df_sample.iloc[999]
print("\nNew RSI Features:")
rsi_cols = ['rsi14_zone', 'rsi7_zone', 'rsi14_ma_div', 'rsi7_ma_div', 
            'rsi14_momentum', 'rsi7_momentum', 'rsi14_cross', 'rsi7_cross', 
            'rsi_agreement']
for col in rsi_cols:
    print(f"{col:<15}: {sample_row[col]:.6f}")

# Display current columns
print("\nFinal column list:")
print(df_sample.columns.tolist())


Sample row (index 999) after RSI feature engineering:

New RSI Features:
rsi14_zone     : 1.000000
rsi7_zone      : 1.000000
rsi14_ma_div   : 0.150355
rsi7_ma_div    : -0.018707
rsi14_momentum : 0.000000
rsi7_momentum  : 0.000000
rsi14_cross    : 0.000000
rsi7_cross     : 0.000000
rsi_agreement  : 1.000000

Final column list:
['Pivot Highs', 'Pivot Lows', 'Pivot High Mistaken', 'Pivot Low Mistaken', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_upper', 'BB_middle', 'BB_lower', 'sr_below_1', 'sr_below_2', 'sr_below_3', 'sr_above_1', 'sr_above_2', 'sr_above_3', 'sr_weight_1', 'sr_weight_2', 'sr_weight_3', 'num_sr_below', 'num_sr_above', 'total_weight_below', 'total_weight_above', 'tl_below_1', 'tl_below_2', 'tl_below_3', 'tl_above_1', 'tl_above_2', 'tl_above_3', 'tl_weight_1', 'tl_weight_2', 'tl_weight_3', 'num_tl_below', 'num_tl_above', 'total_tl_weight_below', 'total_tl_weight_above', 'month', 'day', 'day_of_week', 'hour', 'season', 'open', 'high', 'low